In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

%matplotlib inline

In [2]:
def find_nearest(array, value):
    idx = (np.abs(array - value)).argmin()
    return idx

In [3]:
def pre_rip_sequence(signal,ts,rip_times,n_steps):
    '''
    Get segments of signal of size n_steps prior to the onset of a ripple
    '''
    X = list()
    for i in range(len(rip_times)): 
        rip_ind = find_nearest(ts,rip_times[i][0])
        if rip_ind > n_steps:
            #Create pre sequence
            seq_x = signal[rip_ind-n_steps:rip_ind] 
            X.append(seq_x)
    return np.array(X)

In [4]:
def ripple_ind(rip_times,ts):
    '''
    Finds the ts index of each ripple start time
    '''
    rip_inds = np.zeros(len(rip_times),dtype='int')
    for irip in range(len(rip_times)):
        rip_inds[irip] = find_nearest(ts,rip_times[irip][0])
    
    return rip_inds

In [5]:
def non_rip_seq(signal,ts,rip_inds,n_steps,n_segs):
    '''
    Generates n_segs of segments of signal of length n_steps that do not precede a ripple onset
    '''
    X = list()
    n2_comp = n_segs #Number of segments left to compute
    while n2_comp > 0:
        #Generates random indices
        rand_ind = np.random.choice(len(ts),n2_comp)
        
        #Checks whether a ripple index appears n_steps prior to or following the candidate index and excludes if so
        for irand in rand_ind:
            samp_x = np.arange(irand-n_steps,irand+n_steps)
            if set(samp_x).isdisjoint(rip_inds):
                X.append(signal[irand-n_steps:irand])
                n2_comp -= 1    
    return X

In [6]:
from scipy import io
import glob, os
import pickle

#    ---    Generates pre-ripple and pre-non ripple LFP segments from data in each .mat file    ----

n_steps = 300 #Step size of each LFP segment

cwd = os.getcwd() #Get path of current directory (if ripple times stored there)
os.chdir(cwd)
for file in glob.glob("*.mat"):
    
    #Load .mat files (generated by MatLab)
    mat = io.loadmat(file)

    #Get data from mat file
    LFP_Samples = mat['LFP_Samples']
    LFP_Samples = LFP_Samples[0]
    LFP_TS = mat['LFP_Timestamps']
    LFP_TS = LFP_TS[0]
    Fs = mat['Fs']
    Rip_Times = mat['Rip_Time_Array']
    Fs = Fs.reshape(1)
    
    #Runs above functions to get each LFP sequence
    X_rip = pre_rip_sequence(LFP_Samples,LFP_TS,Rip_Times,n_steps)
    y_rip = np.ones((len(X_rip))) #Target array (1 specifies ripple present)
    rip_inds = ripple_ind(Rip_Times,LFP_TS)
    X_nrip = non_rip_seq(LFP_Samples,LFP_TS,rip_inds,n_steps,len(X_rip))
    y_nrip = np.zeros((len(X_nrip))) #Target array (0 for no ripple)

    #Creates LFP_Segs array to add each segment to if it doesn't exist
    if 'LFP_Segs' not in globals():
        LFP_Segs = X_rip
        Rip_Val = y_rip
        LFP_Segs = np.append(LFP_Segs,X_nrip,axis=0)
        Rip_Val = np.append(Rip_Val,y_nrip,axis=0)
    else:
        LFP_Segs = np.append(LFP_Segs,X_rip,axis=0)
        Rip_Val = np.append(Rip_Val,y_rip,axis=0)
        LFP_Segs = np.append(LFP_Segs,X_nrip,axis=0)
        Rip_Val = np.append(Rip_Val,y_nrip,axis=0)
        

#Save LFP Segs and target values
with open('LFP_Segs.pkl', 'wb') as f:  
    pickle.dump([LFP_Segs,Rip_Val], f)